In [ ]:
!pip install -U -- pre astroquery

In [2]:
# 引入 astroquery.mpc 模組中的 MPC 類別
from astroquery.mpc import MPC

# 使用 MPC 類別的 query_object() 方法來查詢小行星或彗星的軌道相關資訊，例如軌道傾角 (inclination)、半長軸 (semimajor axis)、離心率 (eccentricity)
# 若想查詢彗星的軌道相關資訊，要將 target_type 參數改為 'comet' 並將 name 參數修改為彗星名稱
target_orbital_info = MPC.query_object(target_type='asteroid', name='Eris')
print(target_orbital_info)
print(f"該天體的軌道傾角 {target_orbital_info[0]['inclination']} 度")
print(f"該天體的半長軸為 {target_orbital_info[0].get('semimajor_axis')} AU")

# 使用 MPC 類別的 get_observatory_codes() 方法取得所有觀測站編碼，並從中取出位於台灣的鹿林天文台的編碼
observatory_codes = MPC.get_observatory_codes()
lulin_obs_code = observatory_codes[observatory_codes['Name'] == 'Lulin Observatory']['Code'][0]
print(f"鹿林天文台的編碼為 {lulin_obs_code}")

# 使用 MPC 類別的 get_ephemeris() 方法來取得目標天體在特定日期範圍內的星曆表，並設定觀測地點為鹿林天文台
# target_ephemeris = MPC.get_ephemeris('Eris', start='2023-09-02', step='1d', location=lulin_obs_code)
# target_ephemeris

[{'absolute_magnitude': '-1.23', 'aphelion_distance': '97.682', 'arc_length': 25633, 'argument_of_perihelion': '150.71789', 'ascending_node': '36.0636902', 'critical_list_numbered_object': False, 'delta_v': 16.0, 'designation': None, 'earth_moid': 37.5858, 'eccentricity': '0.4343069', 'epoch': '2024-10-17.0', 'epoch_jd': '2460600.5', 'first_observation_date_used': '1954-09-03.0', 'first_opposition_used': '1954', 'inclination': '43.77787', 'jupiter_moid': 33.34, 'km_neo': False, 'last_observation_date_used': '2024-11-06.0', 'last_opposition_used': '2024', 'mars_moid': 36.9787, 'mean_anomaly': '210.56661', 'mean_daily_motion': '0.0017537', 'mercury_moid': 38.1275, 'name': 'Eris', 'neo': False, 'number': 136199, 'observations': 2602, 'oppositions': 33, 'orbit_type': 10, 'orbit_uncertainty': '2', 'p_vector_x': '-0.9129665', 'p_vector_y': '-0.34378288', 'p_vector_z': '0.21978515', 'perihelion_date': '2258-02-05.4315670023148148', 'perihelion_date_jd': '2545812.93157', 'perihelion_distance':

In [4]:
# 引入 astroquery.jplhorizons 模組中的 Horizons 類別
from astroquery.jplhorizons import Horizons
# 引入 astropy 的 units 模組，用來處理各種物理單位，如度數、距離等
import astropy.units as u

# 建立一個 ID 為 499 的 Horizons 物件；該 ID 代表火星。
# Horizons 系統網站可以查詢到各個太陽系天體的 ID：https://ssd.jpl.nasa.gov/horizons/app.html
# 然後用該物件的 elements() 方法來查詢火星的軌道參數，例如軌道傾角 (incl)、半長軸 (a)、離心率 (e)
horizons_obj = Horizons(id='499')
target_orbital_info = horizons_obj.elements()
print(target_orbital_info)

# 設定鹿林天文台的地理位置資訊，包括經度、緯度和海拔
lulin_observatory_location = {
    'lon': 120.872624 * u.deg,
    'lat': 23.469447 * u.deg
}
lulin_observatory_location['elevation'] = 2.862 * u.km

# 建立一個 ID 為 599 的 Horizons 物件；該 ID 代表木星。
# 並設定觀測地點為鹿林天文台，以及觀測的時間範圍和間隔
horizons_obj = Horizons(id='599', location=lulin_observatory_location,
                        epochs={'start':'2023-09-02', 'stop':'2023-09-03', 'step':'1h'})
target_ephemeris = horizons_obj.ephemerides()
# target_ephemeris

targetname    datetime_jd    ...         Q                 P        
   ---             d         ...         AU                d        
---------- ----------------- ... ----------------- -----------------
Mars (499) 2460623.073883342 ... 1.666099344364666 687.0462766676367


In [9]:
# 引入必要的模組
from astroquery.jplhorizons import Horizons
import plotly.graph_objects as go
import astropy.units as u

# 建立一個字典，用來儲存各個行星及它們在 Horizons 系統中的 ID
planets = {
    '水星': '199',
    '金星': '299',
    '火星': '499',
    '木星': '599',
    '土星': '699',
    '天王星': '799',
    '海王星': '899'
}

# 設定鹿林天文台的地理位置，包括經度、緯度和海拔
lulin_observatory_location = {
    'lon': 120.872624 * u.deg,
    'lat': 23.469447 * u.deg,
    'elevation': 2.862 * u.km
}

# 設定要查詢星曆表的日期範圍
start_date = '2023-10-08'
end_date = '2024-11-08'

# 產生一個 go.Figure 物件，用於建立和組織圖形結構
fig = go.Figure()

# 迭代 planets 字典中的每一個行星及其 ID
for planet_name, planet_id in planets.items():
    # 使用 Horizons 類別取得行星在指定的觀測地點及日期範圍內的星曆表
    obj = Horizons(id=planet_id, location=lulin_observatory_location,
                   epochs={'start': start_date, 'stop': end_date, 'step': '1d'})
    eph = obj.ephemerides()
    
    # 針對每個資料點，產生顯示於滑鼠懸浮時的文字資訊
    hover_text = [f"日期: {row['datetime_str']}<br>赤經: {row['RA']}<br>赤緯: {row['DEC']}" for row in eph]
    
    # 使用 Figure 物件的 add_trace() 方法在圖中加入各行星的軌跡資料
    fig.add_trace(go.Scatter(
        x=eph['RA'], y=eph['DEC'],
        mode='lines+markers',
        name=planet_name,
        text=hover_text,
        hoverinfo='name+text'
    ))

# 使用 Figure 物件的 update_layout() 方法來更新圖的外觀設定
fig.update_layout(
    title=f'各行星在天球上的軌跡 ({start_date} 至 {end_date})',
    xaxis_title='赤經（度）',
    yaxis_title='赤緯（度）'
)

# 顯示圖表
fig.show()

In [ ]:
# TODO: 使用英文版
from astroquery.jplhorizons import Horizons
import plotly.graph_objects as go
import astropy.units as u

# Create a dictionary to store the names of the planets and their Horizons system IDs
planets = {
    'Mercury': '199',
    'Venus': '299',
    'Mars': '499',
    'Jupiter': '599',
    'Saturn': '699',
    'Uranus': '799',
    'Neptune': '899'
}

# Set the geographical location of Lulin Observatory (longitude, latitude, and elevation)
lulin_observatory_location = {
    'lon': 120.872624 * u.deg,
    'lat': 23.469447 * u.deg,
    'elevation': 2.862 * u.km
}

# Set the date range for querying the ephemeris
start_date = '2023-10-08'
end_date = '2024-11-08'

# Create a figure object for plotting
fig = go.Figure()

# Iterate through the planets dictionary to fetch data for each planet
for planet_name, planet_id in planets.items():
    # Use the Horizons class to get the ephemeris for each planet
    obj = Horizons(id=planet_id, location=lulin_observatory_location,
                   epochs={'start': start_date, 'stop': end_date, 'step': '1d'})
    eph = obj.ephemerides()
    
    # Generate hover text for each data point
    hover_text = [f"Date: {row['datetime_str']}<br>RA: {row['RA']}<br>DEC: {row['DEC']}" for row in eph]
    
    # Add trace to the figure for each planet's trajectory
    fig.add_trace(go.Scatter(
        x=eph['RA'], y=eph['DEC'],
        mode='lines+markers',
        name=planet_name,
        text=hover_text,
        hoverinfo='name+text'
    ))

# Update the layout of the figure, including title, axis labels, and font
fig.update_layout(
    title=f'Planetary Trajectories on the Celestial Sphere ({start_date} to {end_date})',
    xaxis_title='Right Ascension (°)',
    yaxis_title='Declination (°)',
    font=dict(
        family='Times New Roman',
        size=14
    )
)

# Display the plot
fig.show()


In [ ]:
# TODO: 使用英文版
from astroquery.jplhorizons import Horizons
import plotly.graph_objects as go
import astropy.units as u

# Create a dictionary to store the names of the planets and their Horizons system IDs
planets = {
    'Mercury': '199',
    'Venus': '299',
    'Mars': '499',
    'Jupiter': '599',
    'Saturn': '699',
    'Uranus': '799',
    'Neptune': '899'
}

# Set the geographical location of Lulin Observatory (longitude, latitude, and elevation)
lulin_observatory_location = {
    'lon': 120.872624 * u.deg,
    'lat': 23.469447 * u.deg,
    'elevation': 2.862 * u.km
}

# Set the date range for querying the ephemeris
start_date = '2023-10-08'
end_date = '2024-11-08'

# Create a figure object for plotting
fig = go.Figure()

# Iterate through the planets dictionary to fetch data for each planet
for planet_name, planet_id in planets.items():
    # Use the Horizons class to get the ephemeris for each planet
    obj = Horizons(id=planet_id, location=lulin_observatory_location,
                   epochs={'start': start_date, 'stop': end_date, 'step': '1d'})
    eph = obj.ephemerides()
    
    # Generate hover text for each data point
    hover_text = [f"Date: {row['datetime_str']}<br>RA: {row['RA']}<br>DEC: {row['DEC']}" for row in eph]
    
    # Add trace to the figure for each planet's trajectory
    fig.add_trace(go.Scatter(
        x=eph['RA'], y=eph['DEC'],
        mode='lines+markers',
        name=planet_name,
        text=hover_text,
        hoverinfo='name+text'
    ))

# Update the layout of the figure, including title, axis labels, and font
fig.update_layout(
    title=f'Planetary Trajectories on the Celestial Sphere ({start_date} to {end_date})',
    xaxis_title='Right Ascension (°)',
    yaxis_title='Declination (°)',
    font=dict(
        family='Times New Roman',
        size=14
    )
)

# Display the plot
fig.show()


In [ ]:
# TODO: 水星逆行
from astroquery.jplhorizons import Horizons
import plotly.graph_objects as go
import astropy.units as u

# Define the target planet (Mercury) and its Horizons ID
planet_id = '199'
planet_name = 'Mercury'

# Set the location for Lulin Observatory (longitude, latitude, and elevation)
lulin_observatory_location = {
    'lon': 120.872624 * u.deg,
    'lat': 23.469447 * u.deg,
    'elevation': 2.862 * u.km
}

# Set the date range for querying the ephemeris
start_date = '2023-06-01'
end_date = '2024-01-01'

# Fetch the ephemeris data for Mercury
obj = Horizons(id=planet_id, location=lulin_observatory_location,
               epochs={'start': start_date, 'stop': end_date, 'step': '1d'})
eph = obj.ephemerides()

# Initialize lists to store data for plotting
ra_values = eph['RA']
dec_values = eph['DEC']
dates = eph['datetime_str']

# Identify retrograde motion (when RA decreases)
is_retrograde = [ra_values[i] > ra_values[i + 1] for i in range(len(ra_values) - 1)]
is_retrograde.append(is_retrograde[-1])  # Append last value to match length

# Prepare hover text for interactive plot
hover_text = [f"Date: {date}<br>RA: {ra:.2f}<br>DEC: {dec:.2f}" 
              for date, ra, dec in zip(dates, ra_values, dec_values)]

# Create a plotly figure
fig = go.Figure()

# Add trace for normal (prograde) motion
fig.add_trace(go.Scatter(
    x=[ra for ra, retro in zip(ra_values, is_retrograde) if not retro],
    y=[dec for dec, retro in zip(dec_values, is_retrograde) if not retro],
    mode='lines+markers',
    name='Prograde Motion',
    text=[text for text, retro in zip(hover_text, is_retrograde) if not retro],
    hoverinfo='text',
    line=dict(color='blue')
))

# Add trace for retrograde motion
fig.add_trace(go.Scatter(
    x=[ra for ra, retro in zip(ra_values, is_retrograde) if retro],
    y=[dec for dec, retro in zip(dec_values, is_retrograde) if retro],
    mode='lines+markers',
    name='Retrograde Motion',
    text=[text for text, retro in zip(hover_text, is_retrograde) if retro],
    hoverinfo='text',
    line=dict(color='red')
))

# Update layout with title and axis labels
fig.update_layout(
    title=f'{planet_name} Retrograde and Prograde Motion ({start_date} to {end_date})',
    xaxis_title='Right Ascension (°)',
    yaxis_title='Declination (°)',
    font=dict(
        family='Times New Roman',
        size=14
    )
)

# Display the plot
fig.show()